# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.27,37.84,48.0,1922.0,409.0,1026.0,335.0,1.7969,110400.0,NEAR BAY
1,-122.28,37.81,52.0,773.0,143.0,377.0,115.0,2.4083,98200.0,NEAR BAY
2,-122.16,37.75,38.0,2457.0,624.0,1516.0,482.0,1.5625,91700.0,NEAR BAY
3,-122.30,37.87,10.0,503.0,118.0,228.0,100.0,2.1705,150000.0,NEAR BAY
4,-122.15,37.74,52.0,2898.0,557.0,1338.0,550.0,3.8510,183500.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
194,-119.35,36.22,29.0,2051.0,351.0,915.0,343.0,3.1944,106800.0,INLAND
195,-119.35,36.19,6.0,958.0,226.0,734.0,230.0,1.0349,67800.0,INLAND
196,-119.18,34.28,17.0,4526.0,717.0,2088.0,655.0,5.6885,268200.0,NEAR OCEAN
197,-118.77,34.27,10.0,1658.0,310.0,1053.0,333.0,4.7574,209900.0,<1H OCEAN


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StandardScaler, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
train_set = train_set.withColumn('mean_bedrooms_households', train_set['total_bedrooms'] / train_set['households'])
train_set = train_set.withColumn('mean_rooms_households', train_set['total_rooms'] / train_set['households'])
train_set = train_set.withColumn('mean_population_households', train_set['population'] / train_set['households'])

test_set = test_set.withColumn('mean_bedrooms_households', test_set['total_bedrooms'] / test_set['households'])
test_set = test_set.withColumn('mean_rooms_households', test_set['total_rooms'] / test_set['households'])
test_set = test_set.withColumn('mean_population_households', test_set['population'] / test_set['households'])

In [0]:
train_set = train_set.drop('ocean_proximity', 'total_bedrooms', 'total_rooms')
test_set  = test_set.drop('ocean_proximity', 'total_bedrooms', 'total_rooms')